### Prepare space in flash

In [1]:
import os, subprocess

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220526_P189_laqueus_DiI_L-J30/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, "ants")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Prepare mask

In [4]:
import nrrd
from probemapper.io import save_nifti, load_nifti
import numpy as np

In [5]:
raw = load_nifti(os.path.join(rootdir, "analysis/536BP_10um.nii"))
mask, _ = nrrd.read(os.path.join(rootdir, "analysis/slicer_536BP/Segmentation.seg.nrrd"))
mask = np.swapaxes(mask, 0, 2)
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "536BP_10um_masked.nii"), 10, 10, 10)

In [6]:
raw = load_nifti("/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/SYTOXG_10um.nii")
mask = load_nifti("/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/SYTOXG_10um_mask.nii.gz")
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "O_Laqueues_v0.1_10um_masked.nii"), 10, 10, 10)

### Registration

In [7]:
from probemapper.registration import ANTSParams, ANTSStage, ANTSRegisteration

In [8]:
from probemapper.registration.helper import define_initial_transformation
import numpy as np

aff = np.array([[ 1.0, 0, 0],
                [ 0, 1.0, 0],
                [ 0, 0, 1.0]], dtype='float')
transl = np.array([0, 0, 0], dtype='float')
define_initial_transformation(aff, transl, os.path.join(tmpdir, "initial.mat"))

.mat file saved as /flash/ReiterU/tmp/P189/ants/initial.mat


In [9]:
P = ANTSParams(
    use_float=False,
    interpolation="Linear",
    histogram_matching=False,
    winsorize=[0.05, 1.0],
    init_transform_fx_to_mv=os.path.join(tmpdir, "initial.mat"),
    stages=[
        ANTSStage(
            transform="Affine",
            transform_args="0.1",
            metrics="MI",
            metrics_args="1,128,Regular,0.5",
            convergence="1000x1000x1000,1e-5,15",
            shrink_factors="8x4x2",
            smoothing_sigmas="3x2x1vox"
        ),
        ANTSStage(
            transform="SyN",
            transform_args="0.1,4.0,0.2",
            metrics="CC",
            metrics_args="1,6",
            convergence="500x500x100x30,1e-6,10",
            shrink_factors="8x4x2x1",
            smoothing_sigmas="3x2x1x0vox"
        )
    ]
)

In [10]:
R = ANTSRegisteration(P, ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin", num_threads=12)
R.set_fixed_image(os.path.join(tmpdir, "O_Laqueues_v0.1_10um_masked.nii"), 10)
R.set_moving_image(os.path.join(tmpdir, "536BP_10um_masked.nii"), 10)
R.set_outputs(tmpdir)

In [11]:
R.run()

########################################
['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsRegistration', '--dimensionality', '3', '--initial-moving-transform', '/flash/ReiterU/tmp/P189/ants/initial.mat', '--interpolation', 'Linear', '--use-histogram-matching', '0', '--float', '0', '--verbose', '1', '--output', '[/flash/ReiterU/tmp/P189/ants/F2M_,/flash/ReiterU/tmp/P189/ants/Warped.nii.gz,/flash/ReiterU/tmp/P189/ants/InvWarped.nii.gz]', '--transform', 'Affine[0.1]', '--metric', 'MI[/flash/ReiterU/tmp/P189/ants/O_Laqueues_v0.1_10um_masked.nii,/flash/ReiterU/tmp/P189/ants/536BP_10um_masked.nii,1,128,Regular,0.5]', '--convergence', '[1000x1000x1000,1e-5,15]', '--shrink-factors', '8x4x2', '--smoothing-sigmas', '3x2x1vox', '--transform', 'SyN[0.1,4.0,0.2]', '--metric', 'CC[/flash/ReiterU/tmp/P189/ants/O_Laqueues_v0.1_10um_masked.nii,/flash/ReiterU/tmp/P189/ants/536BP_10um_masked.nii,1,6]', '--convergence', '[500x500x100x30,1e-6,10]', '--shrink-factors', '8x4x2x1', '--smoothing-sigmas', '3x2x1x0vox']


 2DIAGNOSTIC,     4, -2.354222116731e-01, 1.797693134862e+308, 2.8493e+00, 6.2096e-02,
 2DIAGNOSTIC,     5, -2.356666400837e-01, 1.797693134862e+308, 2.9083e+00, 5.8984e-02,
 2DIAGNOSTIC,     6, -2.359275038369e-01, 1.797693134862e+308, 2.9709e+00, 6.2573e-02,
 2DIAGNOSTIC,     7, -2.361398278176e-01, 1.797693134862e+308, 3.0351e+00, 6.4234e-02,
 2DIAGNOSTIC,     8, -2.369017661435e-01, 1.797693134862e+308, 3.0978e+00, 6.2724e-02,
 2DIAGNOSTIC,     9, -2.381678948945e-01, 1.797693134862e+308, 3.1617e+00, 6.3810e-02,
 2DIAGNOSTIC,    10, -2.399194846201e-01, 1.797693134862e+308, 3.2358e+00, 7.4135e-02,
 2DIAGNOSTIC,    11, -2.450679026445e-01, 1.797693134862e+308, 3.3001e+00, 6.4272e-02,
 2DIAGNOSTIC,    12, -2.513286617122e-01, 1.797693134862e+308, 3.3646e+00, 6.4575e-02,
 2DIAGNOSTIC,    13, -2.795910595875e-01, 1.797693134862e+308, 3.4266e+00, 6.1941e-02,
 2DIAGNOSTIC,    14, -2.940760715007e-01, 1.797693134862e+308, 3.4876e+00, 6.0983e-02,
 2DIAGNOSTIC,    15, -3.064389966704e-01, 9

 2DIAGNOSTIC,    14, -3.440062652756e-01, 1.797693134862e+308, 1.8173e+01, 3.9902e-01,
 2DIAGNOSTIC,    15, -3.458613227387e-01, 3.361316385180e-04, 1.8631e+01, 4.5770e-01,
 2DIAGNOSTIC,    16, -3.473117575805e-01, 4.635895814127e-04, 1.9234e+01, 6.0367e-01,
 2DIAGNOSTIC,    17, -3.482406024268e-01, 5.861695502314e-04, 1.9726e+01, 4.9116e-01,
 2DIAGNOSTIC,    18, -3.512177781230e-01, 7.634743655463e-04, 2.0176e+01, 4.5079e-01,
 2DIAGNOSTIC,    19, -3.539095781334e-01, 9.679877063821e-04, 2.0541e+01, 3.6502e-01,
 2DIAGNOSTIC,    20, -3.566822539646e-01, 1.187300907802e-03, 2.1148e+01, 6.0694e-01,
 2DIAGNOSTIC,    21, -3.569587748157e-01, 1.341899155876e-03, 2.1530e+01, 3.8134e-01,
 2DIAGNOSTIC,    22, -3.572327649407e-01, 1.434911130992e-03, 2.2027e+01, 4.9680e-01,
 2DIAGNOSTIC,    23, -3.573288193210e-01, 1.466552893220e-03, 2.2412e+01, 3.8504e-01,
 2DIAGNOSTIC,    24, -3.573472317607e-01, 1.441508945135e-03, 2.2752e+01, 3.4093e-01,
 2DIAGNOSTIC,    25, -3.574486342130e-01, 1.365173512

 2DIAGNOSTIC,   110, -3.705538196362e-01, 8.521782799427e-05, 5.7189e+01, 3.7772e-01,
 2DIAGNOSTIC,   111, -3.707426818115e-01, 7.166717990978e-05, 5.7634e+01, 4.4517e-01,
 2DIAGNOSTIC,   112, -3.707807642585e-01, 5.653094256548e-05, 5.7990e+01, 3.5574e-01,
 2DIAGNOSTIC,   113, -3.708227073433e-01, 3.965722879233e-05, 5.8359e+01, 3.6840e-01,
 2DIAGNOSTIC,   114, -3.710133176747e-01, 2.281759317982e-05, 5.8870e+01, 5.1172e-01,
 2DIAGNOSTIC,   115, -3.710507897537e-01, 1.553737438762e-05, 5.9215e+01, 3.4467e-01,
 2DIAGNOSTIC,   116, -3.710975153873e-01, 1.462854846256e-05, 5.9559e+01, 3.4439e-01,
 2DIAGNOSTIC,   117, -3.712373720319e-01, 1.432519609723e-05, 5.9943e+01, 3.8403e-01,
 2DIAGNOSTIC,   118, -3.716456978609e-01, 1.655534949272e-05, 6.0388e+01, 4.4469e-01,
 2DIAGNOSTIC,   119, -3.718983625833e-01, 1.978408072184e-05, 6.0783e+01, 3.9464e-01,
 2DIAGNOSTIC,   120, -3.720483271463e-01, 2.280132449261e-05, 6.1144e+01, 3.6107e-01,
 2DIAGNOSTIC,   121, -3.729780994353e-01, 2.8917828950

 2DIAGNOSTIC,    22, -3.966525272359e-01, 4.545252379193e-04, 1.3928e+02, 1.9399e+00,
 2DIAGNOSTIC,    23, -3.967497162752e-01, 3.875685517435e-04, 1.4118e+02, 1.8950e+00,
 2DIAGNOSTIC,    24, -3.969513488085e-01, 3.341783207662e-04, 1.4321e+02, 2.0319e+00,
 2DIAGNOSTIC,    25, -3.974587513241e-01, 3.000361558885e-04, 1.4520e+02, 1.9925e+00,
 2DIAGNOSTIC,    26, -3.981619276248e-01, 2.783708683597e-04, 1.4815e+02, 2.9503e+00,
 2DIAGNOSTIC,    27, -3.990536437582e-01, 2.632378331802e-04, 1.5092e+02, 2.7627e+00,
 2DIAGNOSTIC,    28, -3.991140931920e-01, 2.394452926670e-04, 1.5316e+02, 2.2445e+00,
 2DIAGNOSTIC,    29, -3.994877104673e-01, 2.171676249476e-04, 1.5581e+02, 2.6488e+00,
 2DIAGNOSTIC,    30, -3.994904997564e-01, 1.899521555739e-04, 1.5784e+02, 2.0285e+00,
 2DIAGNOSTIC,    31, -3.995073082888e-01, 1.923887848276e-04, 1.5978e+02, 1.9384e+00,
 2DIAGNOSTIC,    32, -3.995510081662e-01, 1.961266402962e-04, 1.6185e+02, 2.0740e+00,
 2DIAGNOSTIC,    33, -3.997274021176e-01, 1.9539644422

  Elapsed time (stage 0): 3.7525e+02


Stage 1
  iterations = 500x500x100x30
  convergence threshold = 1.0000e-06
  convergence window size = 10
  number of levels = 4
  using the CC metric (radius = 6, weight = 1.0000e+00, use gradient filter = 0)
  Shrink factors (level 1 out of 4): [8, 8, 8]
  Shrink factors (level 2 out of 4): [4, 4, 4]
  Shrink factors (level 3 out of 4): [2, 2, 2]
  Shrink factors (level 4 out of 4): [1, 1, 1]
  smoothing sigmas per level: [3, 2, 1, 0]
  Using default NONE metricSamplingStrategy

*** Running SyN registration (varianceForUpdateField = 4.0000e+00, varianceForTotalField = 2.0000e-01) ***

XXDIAGNOSTIC,Iteration,metricValue,convergenceValue,ITERATION_TIME_INDEX,SINCE_LAST
 1DIAGNOSTIC,     1, -7.331871477320e-01, 1.797693134862e+308, 4.8185e+00, 4.8185e+00,
 1DIAGNOSTIC,     2, -7.395089622826e-01, 1.797693134862e+308, 7.0198e+00, 2.2012e+00,
 1DIAGNOSTIC,     3, -7.470871112967e-01, 1.797693134862e+308, 9.3462e+00, 2.3264e+00,
 1DIAGNOSTIC,     4, -

 1DIAGNOSTIC,    88, -8.932346494281e-01, 1.645598818566e-05, 1.9637e+02, 2.1699e+00,
 1DIAGNOSTIC,    89, -8.933782601096e-01, 1.549686844559e-05, 1.9865e+02, 2.2799e+00,
 1DIAGNOSTIC,    90, -8.936422414951e-01, 1.487549772277e-05, 2.0088e+02, 2.2277e+00,
 1DIAGNOSTIC,    91, -8.937621297376e-01, 1.403721562676e-05, 2.0326e+02, 2.3869e+00,
 1DIAGNOSTIC,    92, -8.938740568523e-01, 1.346989616228e-05, 2.0565e+02, 2.3833e+00,
 1DIAGNOSTIC,    93, -8.939999324151e-01, 1.273202556122e-05, 2.0784e+02, 2.1918e+00,
 1DIAGNOSTIC,    94, -8.942359656397e-01, 1.223139172403e-05, 2.1007e+02, 2.2365e+00,
 1DIAGNOSTIC,    95, -8.943520518665e-01, 1.157728390194e-05, 2.1242e+02, 2.3438e+00,
 1DIAGNOSTIC,    96, -8.944536156371e-01, 1.117643999060e-05, 2.1461e+02, 2.1922e+00,
 1DIAGNOSTIC,    97, -8.945565244711e-01, 1.063205718004e-05, 2.1674e+02, 2.1349e+00,
 1DIAGNOSTIC,    98, -8.947841687193e-01, 1.035111030325e-05, 2.1902e+02, 2.2723e+00,
 1DIAGNOSTIC,    99, -8.948672187887e-01, 9.8247021951

 1DIAGNOSTIC,     9, -9.023592022686e-01, 1.797693134862e+308, 5.3304e+02, 1.5594e+01,
 1DIAGNOSTIC,    10, -9.029283632730e-01, 6.177063605535e-04, 5.5014e+02, 1.7102e+01,
 1DIAGNOSTIC,    11, -9.034286613870e-01, 5.045450026549e-04, 5.6533e+02, 1.5186e+01,
 1DIAGNOSTIC,    12, -9.038783705155e-01, 4.100311119696e-04, 5.7991e+02, 1.4582e+01,
 1DIAGNOSTIC,    13, -9.042657038577e-01, 3.342179300345e-04, 5.9594e+02, 1.6028e+01,
 1DIAGNOSTIC,    14, -9.046091209109e-01, 2.735606713048e-04, 6.1262e+02, 1.6678e+01,
 1DIAGNOSTIC,    15, -9.049151694809e-01, 2.248738528994e-04, 6.2981e+02, 1.7186e+01,
 1DIAGNOSTIC,    16, -9.051884922700e-01, 1.856664867128e-04, 6.4553e+02, 1.5725e+01,
 1DIAGNOSTIC,    17, -9.054270375610e-01, 1.540458753226e-04, 6.6174e+02, 1.6206e+01,
 1DIAGNOSTIC,    18, -9.056439312810e-01, 1.286288051393e-04, 6.7611e+02, 1.4375e+01,
 1DIAGNOSTIC,    19, -9.058411870434e-01, 1.081102003090e-04, 6.9075e+02, 1.4636e+01,
 1DIAGNOSTIC,    20, -9.060161909529e-01, 9.134117026

 1DIAGNOSTIC,    23, -9.041439993487e-01, 8.730122650621e-05, 4.2873e+03, 1.1294e+02,
 1DIAGNOSTIC,    24, -9.043857026828e-01, 7.939708726647e-05, 4.3985e+03, 1.1118e+02,
 1DIAGNOSTIC,    25, -9.046148690042e-01, 7.247911839657e-05, 4.5119e+03, 1.1346e+02,
 1DIAGNOSTIC,    26, -9.048385026476e-01, 6.649934878336e-05, 4.6269e+03, 1.1498e+02,
 1DIAGNOSTIC,    27, -9.050538005825e-01, 6.126858842217e-05, 4.7452e+03, 1.1826e+02,
 1DIAGNOSTIC,    28, -9.052620353598e-01, 5.663843471185e-05, 4.8612e+03, 1.1608e+02,
 1DIAGNOSTIC,    29, -9.054599046040e-01, 5.247291415487e-05, 4.9737e+03, 1.1245e+02,
 1DIAGNOSTIC,    30, -9.056474762184e-01, 4.867823507419e-05, 5.0843e+03, 1.1055e+02,
 1DIAGNOSTIC,    31, -9.058247842717e-01, 4.517714989492e-05, 5.1912e+03, 1.0699e+02,
 1DIAGNOSTIC,    32, -9.059932888188e-01, 4.193880115996e-05, 5.3099e+03, 1.1867e+02,
 1DIAGNOSTIC,    33, -9.061538551646e-01, 3.893015754886e-05, 5.4273e+03, 1.1737e+02,
 1DIAGNOSTIC,    34, -9.063063325933e-01, 3.6104115766

 1DIAGNOSTIC,    18, -8.930390086187e-01, 6.742789092484e-05, 2.8310e+04, 7.5881e+02,
 1DIAGNOSTIC,    19, -8.931942316452e-01, 6.168808268405e-05, 2.9070e+04, 7.6060e+02,
 1DIAGNOSTIC,    20, -8.933481593978e-01, 5.670420522584e-05, 2.9826e+04, 7.5571e+02,
 1DIAGNOSTIC,    21, -8.934957582912e-01, 5.236018545714e-05, 3.0582e+04, 7.5651e+02,
 1DIAGNOSTIC,    22, -8.936360081120e-01, 4.850915589866e-05, 3.1343e+04, 7.6108e+02,
 1DIAGNOSTIC,    23, -8.937710961113e-01, 4.502297115011e-05, 3.2102e+04, 7.5880e+02,
 1DIAGNOSTIC,    24, -8.939020964640e-01, 4.184941660727e-05, 3.2854e+04, 7.5174e+02,
 1DIAGNOSTIC,    25, -8.940267992791e-01, 3.889196896351e-05, 3.3604e+04, 7.5024e+02,
 1DIAGNOSTIC,    26, -8.941446432853e-01, 3.611618737868e-05, 3.4359e+04, 7.5458e+02,
 1DIAGNOSTIC,    27, -8.942559172595e-01, 3.345803279130e-05, 3.5097e+04, 7.3774e+02,
 1DIAGNOSTIC,    28, -8.943612123677e-01, 3.094058084099e-05, 3.5829e+04, 7.3284e+02,
 1DIAGNOSTIC,    29, -8.944592304051e-01, 2.8527988885

### Copy files to bucket

In [12]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/P189/ants', 'deigo:/bucket/ReiterU/DBS/data/20220526_P189_laqueus_DiI_L-J30/analysis'], returncode=0)